In [ ]:
import math
import numpy as np

from common import *
from mcs_shared import load_day, load_flight, load_model, load_topo

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from holoviews import opts, Cycle

# Dates

In [ ]:
peak_als_depth = ["20230209", "20230316", "20240115"]
flights_feb = ["20220217", "20230209"]
flights_2024 = ["20231228", "20240115", "20240315"]
mcs_all = ["20220217", "20220317", "20230209", "20230316", "20240115", "20240315"]
mcs_full_domain =["20220217", "20230209", "20230316", "20231228", "20240115", "20240315"]
# "20210310", has the smallest area of all March month
# "20231113", too early low snow, no pattern established
# "20240213", Flight line issue

## Get common area across all flights

In [ ]:
def common_area_factors(flights):
    als_depth = []
    masks = []

    for flight in flights:
        als, mask = load_flight(flight, masked=False)
        
        als_depth.append(als)
        masks.append(mask.values)

    joined_mask = np.logical_and.reduce(np.stack(masks))

    topo = load_topo(joined_mask)
    
    als_area = []
    als_factors = []
    als_depth_masked = []
    
    for als in als_depth:
        als.coords['mask'] = (('y', 'x'), joined_mask)
        # To exclude the road, add: .where(als.y > 4866255, drop=True)
        als = als.where(als.mask, drop=True)

        als_depth_masked.append(als)
        
        mean_model = float(als.snowdepth.mean().values)
        als = (als.snowdepth / mean_model)

        als_area.append(als)

        als_f = als.values.flatten()
        als_f = als_f[~np.isnan(als_f)]

        als_factors.append(als_f)

    return als_factors, als_depth_masked, als_area, topo

In [ ]:
factors, depths, area_factors, topo = common_area_factors(peak_als_depth)

## Depths

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig, ax = plt.subplots(dpi=300, figsize=(10,4))

pd.DataFrame(depths[0].snowdepth.values.flatten(), columns=[pd.to_datetime(peak_als_depth[0]).strftime('%Y-%m-%d')]).plot(
    kind='density', ax=ax, lw=1, color='royalblue', alpha=0.8
)
pd.DataFrame(depths[1].snowdepth.values.flatten(), columns=[pd.to_datetime(peak_als_depth[1]).strftime('%Y-%m-%d')]).plot(
    kind='density', ax=ax, lw=1, color='peru', alpha=0.8
)
pd.DataFrame(depths[2].snowdepth.values.flatten(), columns=[pd.to_datetime(peak_als_depth[2]).strftime('%Y-%m-%d')]).plot(
    kind='density', ax=ax, lw=1, color='teal', alpha=0.8
)
ax.set_xlim(0,4.5)
ax.set_xlabel('Snow Depth (m)')
ax.set_ylim(-0.05, 1.6)

In [ ]:
fig, ax = plt.subplots(dpi=300, figsize=(10,4))

pd.DataFrame(factors[0], columns=[pd.to_datetime(peak_als_depth[0]).strftime('%Y-%m-%d')]).plot(
    kind='density', ax=ax, lw=1, color='royalblue', alpha=0.8
)
pd.DataFrame(factors[1], columns=[pd.to_datetime(peak_als_depth[1]).strftime('%Y-%m-%d')]).plot(
    kind='density', ax=ax, lw=1, color='peru', alpha=0.8
)
pd.DataFrame(factors[2], columns=[pd.to_datetime(peak_als_depth[2]).strftime('%Y-%m-%d')]).plot(
    kind='density', ax=ax, lw=1, color='teal', alpha=0.8
)
ax.set_xlim(0,4.5)
ax.set_xlabel('Factors')
ax.set_ylim(-0.05, 1.6)

In [ ]:
jan_feb = area_factors[2] - area_factors[0]
feb_march = area_factors[0] - area_factors[1]

In [ ]:
fig, (ax, ax2) = plt.subplots(ncols=2, dpi=300, figsize=(10,4), sharey=True, width_ratios=[4.5, 5.5])
jan_feb.plot(ax=ax, add_colorbar=False, clim=(-.1, .1), vmin=-0.5, vmax=0.5, cmap='RdBu', )
feb_march.plot(ax=ax2, vmin=-0.5, vmax=0.5, cmap='RdBu', cbar_kwargs={'label': r'$\Delta$ Factors'})

ax.set_title('')
ax2.set_title('')

ax.set_yticklabels([])
ax.set_xticklabels([])
ax2.set_xticklabels([])

ax.set_ylabel('')
ax.set_xlabel('')
ax2.set_ylabel('')
ax2.set_xlabel('')

fig.tight_layout()

In [ ]:
def reduce_nan_xarray(variable):
    flattened = variable.values.flatten()
    return flattened[~np.isnan(flattened)]

In [ ]:
elevation = topo.dem.values.flatten()
elevation = elevation[~np.isnan(elevation)]

In [ ]:
b, a = np.polyfit(factors[2], elevation, deg=1)
b2, a2 = np.polyfit(factors[0], elevation, deg=1)
b3, a3 = np.polyfit(factors[1], elevation, deg=1)

In [ ]:
fig, axes = plt.subplots(ncols=4, dpi=300, figsize=(10,4.5), sharex=True, sharey=True)
axes[0].scatter(
    factors[2], elevation, c=reduce_nan_xarray(depths[2].snowdepth), 
    s=0.8, cmap='cividis', clim=(0, 4)
)
axes[0].set_title(label=pd.to_datetime(peak_als_depth[2]).strftime('%Y-%m-%d'))
axes[1].scatter(
    factors[0], elevation, c=reduce_nan_xarray(depths[0].snowdepth), 
    s=0.8, cmap='cividis', clim=(0, 4)
)
axes[1].set_title(label=pd.to_datetime(peak_als_depth[0]).strftime('%Y-%m-%d'))
cmap = axes[2].scatter(
    factors[1], elevation, c=reduce_nan_xarray(depths[1].snowdepth), 
    s=0.8, cmap='cividis', clim=(0, 4)
)
axes[2].set_title(label=pd.to_datetime(peak_als_depth[1]).strftime('%Y-%m-%d'))

axes[0].set_ylim(1580, 2500)
axes[0].set_xlim(0, 3)
axes[0].set_ylabel('Elevation (m)')
for ax in axes:
    ax.set_xlabel('Factors')

x_val = np.linspace(0, 2, num=40)

axes[3].plot(
    x_val, a + b * x_val, 
    label=pd.to_datetime(peak_als_depth[2]).strftime('%Y-%m-%d'),
    color='teal', lw=1.5, alpha=0.8, ls='--'
)
axes[3].plot(
    x_val, a2 + b2 * x_val, 
    label=pd.to_datetime(peak_als_depth[0]).strftime('%Y-%m-%d'),
    color='royalblue', lw=1.5, alpha=0.8, ls='--'
)
axes[3].plot(
    x_val, a3 + b3 * x_val, 
    label=pd.to_datetime(peak_als_depth[1]).strftime('%Y-%m-%d'),
    color='peru', lw=1.5, alpha=0.8, ls='--'
)
axes[3].legend(prop={'size': 8})
fig.colorbar(cmap, ax=axes, orientation='horizontal', fraction=.05, label='Snow Depth (m)')

# fig.tight_layout()

## Holoviews

In [ ]:
use_hvplot()

In [ ]:
def get_factors(dates):
    factors = []
    masks = []

    for flight in dates:
        als, mask = load_flight(flight, masked=False)

        masks.append(mask.values)

    joined_mask = np.logical_and.reduce(np.stack(masks))
    
    for idx, flight in enumerate(dates):
        isnobal = load_model(flight)

        isnobal.coords['mask'] = (('y', 'x'), joined_mask)
        isnobal = isnobal.where(isnobal.mask, drop=True)

        depths[idx].coords['mask'] = (('y', 'x'), joined_mask)
        depths[idx] = depths[idx].where(depths[idx].mask, drop=True)

        depths[idx] = depths[idx].snowdepth.values.flatten()
        depths[idx] = depths[idx][~np.isnan(depths[idx])]
        
        factor = isnobal.thickness.values.flatten()
        factor = factor[~np.isnan(factor)]

        factors.append(factor)

    return means, factors, depths 

In [ ]:
def plot_flights(flights, mcs_als, title):
    return hv.Layout([
            hv.NdOverlay(
                { flights[n]: hv.Distribution(mcs_als[n]) for n in range(0, len(mcs_als))}
            ).opts(
                opts.Distribution(filled=False, line_color=Cycle())
            ).opts(**HV_PLOT_OPTS, title='ALS', xlim=(0, 4))
        ]).opts(shared_axes=False, title=title).cols(1)

In [ ]:
mcs_als, depths, area_factors, topo = common_area_factors(peak_als_depth)

In [ ]:
plot_flights(peak_als_depth, mcs_als, "Peak ALS Depth")

# Factors for accumulation period flights 

In [ ]:
def get_factors(dates):
    means = []
    factors = []
    depths = []

    masks = []

    for flight in dates:
        als, mask = load_flight(flight, masked=False)
        mean = float(als.snowdepth.mean().values)

        depths.append(als)
        means.append(mean)
        masks.append(mask.values)

    joined_mask = np.logical_and.reduce(np.stack(masks))
    
    for idx, flight in enumerate(dates):
        isnobal = load_model(flight)

        isnobal.coords['mask'] = (('y', 'x'), joined_mask)
        isnobal = isnobal.where(isnobal.mask, drop=True)

        depths[idx].coords['mask'] = (('y', 'x'), joined_mask)
        depths[idx] = depths[idx].where(depths[idx].mask, drop=True)

        depths[idx] = depths[idx].snowdepth.values.flatten()
        depths[idx] = depths[idx][~np.isnan(depths[idx])]
        
        factor = isnobal.thickness.values.flatten()
        factor = factor[~np.isnan(factor)]

        factors.append(factor)

    return means, factors, depths 

In [ ]:
def plot_flights(flights, title):
    means, factors, depths = get_factors(flights)

    for index, flight in enumerate(means):
        print(f"{flights[index]}: {means[index]}")
        print(f"                  {depths[index].shape}")

    return hv.Layout([
            hv.NdOverlay(
                { flights[n]: hv.Distribution(factors[n]) for n in range(0, len(flights))}
            ).opts(
                opts.Distribution(filled=False, line_color=Cycle())
            ).opts(**HV_PLOT_OPTS, title='Factors'),
            hv.NdOverlay(
                { flights[n]: hv.Distribution(depths[n]) for n in range(0, len(flights))}
            ).opts(
                opts.Distribution(filled=False, line_color=Cycle())
            ).opts(**HV_PLOT_OPTS, title='Depths')
        ]).opts(shared_axes=False, title=title).cols(1)

In [ ]:
plot_flights(peak_als_depth, "Peak ALS Depth")

## February

In [ ]:
plot_flights(flights_feb, "February")

## 2024

In [ ]:
plot_flights(flights_2024, "2024")

# ALL

In [ ]:
plot_flights(mcs_all, "All")

## Gamma

Plot CDF for depths 

In [ ]:
from scipy import stats
import pprint
np.set_printoptions(precision=3, suppress=True)

In [ ]:
means, factors, depths = get_factors(mcs_all)

In [ ]:
def fit_norm(flight, idx, x):
    param = stats.norm.fit(flight)
    return stats.norm.cdf(x, *param)

In [ ]:
def fit_gamma(flight, idx, x):
    param = stats.gamma.fit(flight)
    return stats.gamma.cdf(x, *param)    

In [ ]:
fig, ax = plt.subplots(dpi=300)

for idx, flight in enumerate(factors):
    ecdf_f = stats.ecdf(flight)
    ecdf_f.cdf.plot(ax, label=f'{mcs_all[idx]} ecdf')
    # print(peak_als_depth[idx])
    # pprint.pp(ecdf_f.cdf.quantiles)

ax.legend()
plt.show()

In [ ]:
fig, ax = plt.subplots(dpi=300)

for idx, flight in enumerate(depths):
    ecdf_f = stats.ecdf(flight)
    ecdf_f.cdf.plot(ax, label=f'{mcs_all[idx]} ecdf')
    # print(peak_als_depth[idx])
    # pprint.pp(ecdf_f.cdf.quantiles)

ax.legend()
plt.show()